<h2>Goal:</h2><br>Load NCAA player boxscore data into a database table.

Import libraries

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import getpass
import psycopg2
from psycopg2.extensions import adapt, register_adapter, AsIs
pd.set_option('display.max_columns', 500)

Read in data.

In [2]:
player_boxes = pd.read_csv('../Data/ncaa_player_boxscores.csv')

Inspect the data.

In [3]:
player_boxes.shape

(1035117, 40)

In [4]:
player_boxes.head()

,player_id,assist_percentage,assists,block_percentage,blocks,defensive_rating,defensive_rebound_percentage,defensive_rebounds,effective_field_goal_percentage,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,minutes_played,offensive_rating,offensive_rebound_percentage,offensive_rebounds,personal_fouls,points,steal_percentage,steals,three_point_attempt_rate,three_point_attempts,three_point_percentage,three_pointers,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,two_point_attempts,two_point_percentage,two_pointers,usage_percentage,boxscore,team_abbr,venue
0,dedric-lawson-1,24.5,6,5.8,2,100.0,34.0,10,0.278,18,0.278,5,0.667,12,0.833,10,38,106.0,9.8,4,2,20,2.4,2,0.111,2.0,0.0,0.0,19.9,14,0.430,7.9,2,16.0,0.313,5.0,28.2,2018-11-06-19-kansas,kansas,Home
1,devon-dotson-1,14.7,3,0.0,0,104.0,7.6,2,0.682,11,0.545,6,0.182,2,0.500,1,34,98.0,0.0,0,3,16,4.1,3,0.455,5.0,0.6,3.0,3.2,2,0.673,25.2,4,6.0,0.500,3.0,19.8,2018-11-06-19-kansas,kansas,Home
2,lagerald-vick-1,12.6,3,0.0,0,102.0,12.1,3,0.143,7,0.143,1,0.000,0,NaN,0,32,44.0,0.0,0,2,2,4.3,3,0.571,4.0,0.0,0.0,5.1,3,0.143,22.2,2,3.0,0.333,1.0,11.9,2018-11-06-19-kansas,kansas,Home
3,quentin-grimes-1,24.6,4,0.0,0,110.0,4.3,1,0.714,14,0.500,7,0.143,2,0.500,1,30,134.0,0.0,0,3,21,1.5,1,0.714,10.0,0.6,6.0,1.8,1,0.706,6.3,1,4.0,0.250,1.0,22.4,2018-11-06-19-kansas,kansas,Home
4,udoka-azubuike-1,11.8,1,22.2,4,93.0,6.5,1,0.700,10,0.700,7,0.700,7,0.429,3,20,112.0,9.3,2,4,17,2.3,1,0.000,0.0,NaN,0.0,8.1,3,0.650,13.3,2,10.0,0.700,7.0,31.9,2018-11-06-19-kansas,kansas,Home


Establish database connection.

In [5]:
mypasswd = getpass.getpass()
conn = psycopg2.connect(database = 'cs20_group4',
                              user = 'rodk4',
                              host = 'pgsql.dsa.lan',
                              password = mypasswd)

········


Define cursor that will be used to read and write to and from the database.

In [6]:
cursor = conn.cursor()

Query the table `ncaa_simple_boxscores` to get a dataframe containing boxscores and seasons.

In [7]:
seasons = pd.read_sql_query("""
SELECT
    box.boxscore
    , box.season
FROM ncaa_simple_boxscores box
""",con=conn)

In [8]:
seasons.shape

(276987, 2)

In [9]:
player_boxes.shape

(1035117, 40)

Merge `player_boxes` and `seasons` into one dataframe.

In [10]:
player_boxes = player_boxes.merge(seasons,on='boxscore')

Convert percentages to decimals.

In [11]:
player_boxes.assist_percentage = player_boxes.assist_percentage / 100
player_boxes.block_percentage = player_boxes.block_percentage / 100
player_boxes.defensive_rebound_percentage = player_boxes.defensive_rebound_percentage / 100
player_boxes.offensive_rebound_percentage = player_boxes.offensive_rebound_percentage / 100
player_boxes.steal_percentage = player_boxes.steal_percentage / 100
player_boxes.total_rebound_percentage = player_boxes.total_rebound_percentage / 100
player_boxes.turnover_percentage = player_boxes.turnover_percentage / 100
player_boxes.usage_percentage = player_boxes.usage_percentage / 100

Create three columns that, for each game, track how many games a player has played in a season up to that point, how many games a player has played in their career up to that point, and how many games a player has played for their team up to that point. 

In [12]:
player_boxes['player_game_number_season'] = player_boxes.groupby(['player_id','season']).cumcount() + 1
player_boxes['player_game_number_career'] = player_boxes.groupby(['player_id']).cumcount() + 1
player_boxes['player_game_number_team'] = player_boxes.groupby(['player_id','team_abbr']).cumcount() + 1

In [13]:
player_boxes.head()

,player_id,assist_percentage,assists,block_percentage,blocks,defensive_rating,defensive_rebound_percentage,defensive_rebounds,effective_field_goal_percentage,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,minutes_played,offensive_rating,offensive_rebound_percentage,offensive_rebounds,personal_fouls,points,steal_percentage,steals,three_point_attempt_rate,three_point_attempts,three_point_percentage,three_pointers,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,two_point_attempts,two_point_percentage,two_pointers,usage_percentage,boxscore,team_abbr,venue,season,player_game_number_season,player_game_number_career,player_game_number_team
0,dedric-lawson-1,0.245,6,0.058,2,100.0,0.340,10,0.278,18,0.278,5,0.667,12,0.833,10,38,106.0,0.098,4,2,20,0.024,2,0.111,2.0,0.0,0.0,0.199,14,0.430,0.079,2,16.0,0.313,5.0,0.282,2018-11-06-19-kansas,kansas,Home,2019.0,1,1,1
1,devon-dotson-1,0.147,3,0.000,0,104.0,0.076,2,0.682,11,0.545,6,0.182,2,0.500,1,34,98.0,0.000,0,3,16,0.041,3,0.455,5.0,0.6,3.0,0.032,2,0.673,0.252,4,6.0,0.500,3.0,0.198,2018-11-06-19-kansas,kansas,Home,2019.0,1,1,1
2,lagerald-vick-1,0.126,3,0.000,0,102.0,0.121,3,0.143,7,0.143,1,0.000,0,NaN,0,32,44.0,0.000,0,2,2,0.043,3,0.571,4.0,0.0,0.0,0.051,3,0.143,0.222,2,3.0,0.333,1.0,0.119,2018-11-06-19-kansas,kansas,Home,2019.0,1,1,1
3,quentin-grimes-1,0.246,4,0.000,0,110.0,0.043,1,0.714,14,0.500,7,0.143,2,0.500,1,30,134.0,0.000,0,3,21,0.015,1,0.714,10.0,0.6,6.0,0.018,1,0.706,0.063,1,4.0,0.250,1.0,0.224,2018-11-06-19-kansas,kansas,Home,2019.0,1,1,1
4,udoka-azubuike-1,0.118,1,0.222,4,93.0,0.065,1,0.700,10,0.700,7,0.700,7,0.429,3,20,112.0,0.093,2,4,17,0.023,1,0.000,0.0,NaN,0.0,0.081,3,0.650,0.133,2,10.0,0.700,7.0,0.319,2018-11-06-19-kansas,kansas,Home,2019.0,1,1,1


Extract date from the `boxscore` column to create a new data column.

In [14]:
player_boxes['date'] = pd.to_datetime(player_boxes['boxscore'].str[:10])

For each player, calculate how many days were in between each game he has played. If it is the first game of their career, then assume 150 days has passed between games. (Estimated time since previous game at a lower level of play)

In [15]:
p_ids = []
bx = []
days = []
for player in player_boxes.player_id.unique():
    df = player_boxes.loc[player_boxes.player_id==player].reset_index(drop=True).sort_values(by='date')
    for n in range(len(df)):
        p_ids.append(player)
        bx.append(df.boxscore[n])
        if n == 0:
            days.append(150)
        else:
            try:
                days.append(round((df.date[n] - df.date[n-1])/np.timedelta64(1,'D')))
            except:
                days.append(-1)

Use the lists created in the pervious cell to create a dataframe that contains the column `days_since_last_game`.  Merge this dataframe with `player_boxes` to add the new column.

In [16]:
days_bw_games = pd.DataFrame({'player_id':p_ids,
                             'boxscore':bx,
                             'days_since_last_game':days})

In [17]:
player_boxes = player_boxes.merge(days_bw_games,on=['player_id','boxscore'])

Inspect the dataframe

In [18]:
player_boxes.head()

,player_id,assist_percentage,assists,block_percentage,blocks,defensive_rating,defensive_rebound_percentage,defensive_rebounds,effective_field_goal_percentage,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,minutes_played,offensive_rating,offensive_rebound_percentage,offensive_rebounds,personal_fouls,points,steal_percentage,steals,three_point_attempt_rate,three_point_attempts,three_point_percentage,three_pointers,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,two_point_attempts,two_point_percentage,two_pointers,usage_percentage,boxscore,team_abbr,venue,season,player_game_number_season,player_game_number_career,player_game_number_team,date,days_since_last_game
0,dedric-lawson-1,0.245,6,0.058,2,100.0,0.340,10,0.278,18,0.278,5,0.667,12,0.833,10,38,106.0,0.098,4,2,20,0.024,2,0.111,2.0,0.0,0.0,0.199,14,0.430,0.079,2,16.0,0.313,5.0,0.282,2018-11-06-19-kansas,kansas,Home,2019.0,1,1,1,2018-11-06,150
1,devon-dotson-1,0.147,3,0.000,0,104.0,0.076,2,0.682,11,0.545,6,0.182,2,0.500,1,34,98.0,0.000,0,3,16,0.041,3,0.455,5.0,0.6,3.0,0.032,2,0.673,0.252,4,6.0,0.500,3.0,0.198,2018-11-06-19-kansas,kansas,Home,2019.0,1,1,1,2018-11-06,150
2,lagerald-vick-1,0.126,3,0.000,0,102.0,0.121,3,0.143,7,0.143,1,0.000,0,NaN,0,32,44.0,0.000,0,2,2,0.043,3,0.571,4.0,0.0,0.0,0.051,3,0.143,0.222,2,3.0,0.333,1.0,0.119,2018-11-06-19-kansas,kansas,Home,2019.0,1,1,1,2018-11-06,150
3,quentin-grimes-1,0.246,4,0.000,0,110.0,0.043,1,0.714,14,0.500,7,0.143,2,0.500,1,30,134.0,0.000,0,3,21,0.015,1,0.714,10.0,0.6,6.0,0.018,1,0.706,0.063,1,4.0,0.250,1.0,0.224,2018-11-06-19-kansas,kansas,Home,2019.0,1,1,1,2018-11-06,150
4,udoka-azubuike-1,0.118,1,0.222,4,93.0,0.065,1,0.700,10,0.700,7,0.700,7,0.429,3,20,112.0,0.093,2,4,17,0.023,1,0.000,0.0,NaN,0.0,0.081,3,0.650,0.133,2,10.0,0.700,7.0,0.319,2018-11-06-19-kansas,kansas,Home,2019.0,1,1,1,2018-11-06,150


Write the dataframe to a csv. Following this, reset the kernel, and run the remaining cells. This is to clear memory so that the remaining cells will run successfully.

In [19]:
player_boxes.to_csv('../Data/staging_player_boxes.csv')

<h3>Restart the kernel and run from here down</h3>

In [ ]:
import pandas as pd
import getpass
import psycopg2
from psycopg2.extensions import adapt, register_adapter, AsIs

In [ ]:
player_boxes = pd.read_csv('../Data/staging_player_boxes.csv')

In [ ]:
mypasswd = getpass.getpass()
conn = psycopg2.connect(database = 'cs20_group4',
                              user = 'mwkmr',
                              host = 'pgsql.dsa.lan',
                              password = mypasswd)

In [ ]:
cursor = conn.cursor()

Write the table creation sql

In [ ]:
create_table = """
DROP TABLE IF EXISTS ncaa_player_boxscores;
CREATE TABLE ncaa_player_boxscores (
player_id varchar(50)
, assist_percentage numeric
, assists numeric
, block_percentage numeric
, blocks numeric
, defensive_rating numeric
, defensive_rebound_percentage numeric
, defensive_rebounds numeric
, effective_field_goal_percentage numeric
, field_goal_attempts numeric
, field_goal_percentage numeric
, field_goals numeric
, free_throw_attempt_rate numeric
, free_throw_attempts numeric
, free_throw_percentage numeric
, free_throws numeric
, minutes_played numeric
, offensive_rating numeric
, offensive_rebound_percentage numeric
, offensive_rebounds numeric
, personal_fouls numeric
, points numeric
, steal_percentage numeric
, steals numeric
, three_point_attempt_rate numeric
, three_point_attempts numeric
, three_point_percentage numeric
, three_pointers numeric
, total_rebound_percentage numeric
, total_rebounds numeric
, true_shooting_percentage numeric
, turnover_percentage numeric
, turnovers numeric
, two_point_attempts numeric
, two_point_percentage numeric
, two_pointers numeric
, usage_percentage numeric
, boxscore varchar(100)
, team_abbr varchar(100)
, venue varchar(250)
,season numeric
--
, player_game_number_season numeric
, player_game_number_career numeric
, player_game_number_team numeric
, date TIMESTAMP
, days_since_last_game numeric

)
"""

Execute the table creation sql.

In [ ]:
cursor.execute(create_table)

In [ ]:
conn.commit()

In [ ]:
#conn = psycopg2.connect(user = 'postgres',
#                       host = 'localhost',
#                       password = 'Chaminade#316',
#                       port='5432',
#                       database='basketball')

In [ ]:
#cursor = conn.cursor()

Insert data into newly created table.

In [ ]:
stats = player_boxes.where(pd.notnull(player_boxes),None)
stats = stats.drop(columns='Unnamed: 0')

INSERT_SQL = 'INSERT INTO ncaa_player_boxscores'
INSERT_SQL += '(player_id'
INSERT_SQL += ', assist_percentage'
INSERT_SQL += ', assists'
INSERT_SQL += ', block_percentage'
INSERT_SQL += ', blocks'
INSERT_SQL += ', defensive_rating'
INSERT_SQL += ', defensive_rebound_percentage'
INSERT_SQL += ', defensive_rebounds'
INSERT_SQL += ', effective_field_goal_percentage'
INSERT_SQL += ', field_goal_attempts'
INSERT_SQL += ', field_goal_percentage'
INSERT_SQL += ', field_goals'
INSERT_SQL += ', free_throw_attempt_rate'
INSERT_SQL += ', free_throw_attempts'
INSERT_SQL += ', free_throw_percentage'
INSERT_SQL += ', free_throws'
INSERT_SQL += ', minutes_played'
INSERT_SQL += ', offensive_rating'
INSERT_SQL += ', offensive_rebound_percentage'
INSERT_SQL += ', offensive_rebounds'
INSERT_SQL += ', personal_fouls'
INSERT_SQL += ', points'
INSERT_SQL += ', steal_percentage'
INSERT_SQL += ', steals'
INSERT_SQL += ', three_point_attempt_rate'
INSERT_SQL += ', three_point_attempts'
INSERT_SQL += ', three_point_percentage'
INSERT_SQL += ', three_pointers'
INSERT_SQL += ', total_rebound_percentage'
INSERT_SQL += ', total_rebounds'
INSERT_SQL += ', true_shooting_percentage'
INSERT_SQL += ', turnover_percentage'
INSERT_SQL += ', turnovers'
INSERT_SQL += ', two_point_attempts'
INSERT_SQL += ', two_point_percentage'
INSERT_SQL += ', two_pointers'
INSERT_SQL += ', usage_percentage'
INSERT_SQL += ', boxscore'
INSERT_SQL += ', team_abbr'
INSERT_SQL += ', venue'
INSERT_SQL += ', season'
INSERT_SQL += ', player_game_number_season'
INSERT_SQL += ', player_game_number_career'
INSERT_SQL += ', player_game_number_team'
INSERT_SQL += ', date'
INSERT_SQL +=', days_since_last_game) VALUES'
INSERT_SQL += '(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'
INSERT_SQL += '%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,      %s,%s,%s,%s,%s,%s)'

with conn, conn.cursor() as cursor:
    for row in stats.itertuples(index=False, name=None):
        cursor.execute(INSERT_SQL,row)